# Prepare Wiki Texts

Takes abstracts of English Wikipedia articles from the HuggingFace portal, repackaging them into Parquet files, and vectorizing all the abstract with the E5 model, resulting in 768-dimensional embeddings, and a USearch index on top of them. The outputs are:

- `./data/ann-wiki-6m/title_abstract.parquet`
- `./data/ann-wiki-6m/abstract.e5-base-v2.fbin`
- `./data/ann-wiki-6m/abstract.e5-base-v2.usearch`

In [ ]:
import numpy as np
from datasets import load_dataset
from prepare_embeddings import vectorize_e5, vectorize_uform

In [ ]:
data = load_dataset("wikipedia", "20220301.en", cache_dir="./data/ann-wiki-6m/")
data, data["train"][0]

In [ ]:
df = data["train"].to_pandas()
df['abstract'] = df['text']
df.pop('text')
df.to_parquet('./data/ann-wiki-6m/title_abstract.parquet')

In [ ]:
test_batch = [data["train"][i]["text"] for i in range(10)]
test_batch, vectorize_e5(test_batch)

In [ ]:
from usearch.io import load_matrix, save_matrix
from tqdm import tqdm
import os

In [ ]:
vectors_path = "./data/ann-wiki-6m/abstract.e5-base-v2.fbin"
if not os.path.exists(vectors_path):
    matrix = np.zeros((6458670, 768), dtype=np.float32)
    save_matrix(matrix, vectors_path)

Find all non-zero rows in the NumPy `matrix`, those rows we will need to vectorize.
Iterate through the `data["train"][i]["text"]` in batches, printing progress with `tqdm`.
Vectorize using `vectorize_e5` function, that can take up to 1000 strings-list in a single batch.
Once completed, or if terminated, call `save_matrix(matrix, vectors_path)`, to preserve progress.

In [ ]:
matrix = load_matrix(vectors_path)

In [ ]:
zero_rows = np.any(matrix == 0, axis=1)
zero_indices = np.where(zero_rows)[0]
len(zero_indices) / matrix.shape[0]

In [ ]:
import time

num_entries = len(data["train"])
start_idx = 0
batch_size = 128
last_save_time = time.time()
save_interval = 600  # 10 minutes in seconds

# Using tqdm for progress bar. The "unit_scale" and "unit" arguments allow us to track samples/second.
with tqdm(total=len(zero_indices), unit="samples") as pbar:
    for i in range(0, len(zero_indices), batch_size):
        batch_indices = zero_indices[i:i+batch_size]
        batch_texts = [data["train"][int(idx)]["text"] for idx in batch_indices]

        # Vectorize using vectorize_e5 function
        batch_vectors = vectorize_e5(batch_texts)

        # Update the matrix
        matrix[batch_indices] = batch_vectors

        # Update the progress bar
        pbar.update(len(batch_indices))

        # Check if it's time to save the progress
        current_time = time.time()
        if current_time - last_save_time > save_interval:
            save_matrix(matrix, vectors_path)
            last_save_time = current_time

# Save the matrix after the entire loop is finished
save_matrix(matrix, vectors_path)

In [ ]:
vectors = matrix.astype(np.float16)

In [ ]:
from usearch.index import Index


In [ ]:
index_path = "./data/ann-wiki-6m/abstract.e5-base-v2.usearch"
index = Index(dtype="f16", metric="cos", ndim=768)

if os.path.exists(index_path):
    index.load(index_path)
else:
    batch_size = 1000  # Adjust this based on your preference
    total_batches = int(np.ceil(vectors.shape[0] / batch_size))

    # Using tqdm for progress bar
    for i in tqdm(range(total_batches), desc="Indexing batches"):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, vectors.shape[0])

        batch_keys = np.arange(start_idx, end_idx)
        batch_vectors = vectors[start_idx:end_idx]

        index.add(batch_keys, batch_vectors)

    index.save(index_path)